<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_SB2021_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First things first
### From now on, we will be only using CUDA, a GPU computing API.
### In order to do that, we need to use the GPU runtime from Google Colab that can be selected following the steps in the menu:
### `Runtime > Change Runtime Type > T4 GPU`

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


time: 44.8 s (started: 2023-09-29 00:03:23 +00:00)


In [1]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
use_cuda = True #@param {type:"boolean"}

! rm -f Makefile*
if use_cuda:
  ! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
else:
  ! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile

! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2

In [2]:
#@title Importing pyLiBELa {display-mode: "form"}
import random

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')

pyLiBELa is imported!


In [4]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]
targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()


In [5]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np

Input = PARSER()

# Some default options:
# 1. We are using two processors for Grid calculations;
#

Input.generate_conformers = True;
Input.dock_parallel = True;
Input.parallel_jobs = 2;
Input.write_grids = True
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 150 #@param {type:"number"}
    #atom limit not counting H

# Energy Calculation Parameters:
scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.5 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-5;
Input.dock_min_tol = 1E-4;
search_box = 30.0 #@param {type:"number"}
Input.timeout = 30 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "mma" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
  delta = 2.5 #@param {type:"number"}
  Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
  delta_es = 2.5 #@param {type:"number"}
  Input.deltaij_es6 = pow(delta_es, 6);
  Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;
Input.grid_prefix = "McGrid"

In [11]:
#@title Generating Grids {display-mode: "form"}

%reload_ext autotime
import os


print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

targets_random=random.sample(targets, len(targets));


target_cut = targets[1170:1172]


k=0
for target in target_cut:
  print(k)
  k+=1

  folder = sb_folder+target


  %cd $folder
  subfolder = "self_docking_"+Input.energy_optimizer
  ! mkdir -p $subfolder
  %cd $subfolder

  #!rm -rf *
  if "McGrid.grid" not in os.listdir():

    print()
    print()

    ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
    ! cp ../*.rec.clean.mol2.gz rec.mol2.gz


    if "lig2.smi" not in os.listdir():

      print('Generating SMILES.')
      ! obabel -imol2 lig.mol2.gz -osmi -O lig2.smi &> /dev/null #self docking

    else:

      print('Already SMILEd!')


    insmiles = open('lig2.smi', 'r');
    line=insmiles.readline().strip()
    line2 = line.split()
    ligand_smiles = line2[0]
    insmiles.close();

    lig_src =  'lig.mol2.gz'
    rec_src = 'rec.mol2.gz'

    try:

      REC = Mol2(Input, rec_src)
      RefLig = Mol2(Input, lig_src)
      Lig2 = Mol2()

      ligcheck = Lig2.parse_smiles(Input, ligand_smiles, 'LIG')

      error_message=''

    except (RuntimeError, MemoryError):
      print("Skipping, something went wrong...Check the logs.")
      log = open("log.txt", 'w')
      log.write("Something went wrong, this target was skipped.")
      log.close()
      continue


    if (ligcheck):

      Writer = WRITER(Input)
      Coord = COORD_MC()
      HB = FindHB()
      Ene  = Energy2(Input)

      for i in range(len(REC.residue_pointer)-1):
        HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
      HB.find_ligandHB(lig_src, RefLig);

      Dock = Docker(Writer)
      center = Coord.compute_com(RefLig)


      start_time = timeit.default_timer()
      Grids = Grid(Input, Writer, REC, center)
      time_grid = timeit.default_timer() - start_time

      if not use_cuda:
        print('Not using cuda')

        start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

        grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);


        start_time = timeit.default_timer()
        Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
        time_dock = timeit.default_timer() - start_time


        log = open("log.txt", 'a')

        data = "%.3f\t%.3f\t%.3f\n" %(start_energy,grid_energy, time_dock)
        log.write(data)
        print(data)
        log.close()


      else:

        log = open("log.txt", 'w')

        header = "target\tREC.N\tRefLig.N\tLig2.N\ttime_grid\tstart_energy\tgrid_energy\ttime_dock\n"
        log.write(header)

        data = "%s\t%d\t%d\t%d\t%.3f\t" %(target,REC.N, RefLig.N, Lig2.N,time_grid)
        print(data)
        log.write(data)

        log.close()


  else:
    print('Target already docked!')



#    TARGET  RMSD(Ang)
0
/content/drive/MyDrive/pyLiBELa/SB/9HVP
/content/drive/MyDrive/pyLiBELa/SB/9HVP/self_docking_mma
Target already docked!
1
/content/drive/MyDrive/pyLiBELa/SB/9ICD
/content/drive/MyDrive/pyLiBELa/SB/9ICD/self_docking_mma
Target already docked!
time: 767 ms (started: 2023-09-29 01:12:20 +00:00)


In [12]:
#@title Docking {display-mode: "form"}

%load_ext autotime
import os


print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

targets_random=random.sample(targets, len(targets));

target_cut = targets[700:1172]

k=0
for target in target_cut:
  print(k)
  k+=1

  folder = sb_folder+target


  %cd $folder
  subfolder = "self_docking_"+Input.energy_optimizer
  ! mkdir -p $subfolder
  %cd $subfolder

 # !rm -rf McLiBELa_dock*
  if ("McLiBELa_dock.mol2.gz"  not in os.listdir()) and ("McGrid.grid" in os.listdir()):

    print()
    print()

    ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
    ! cp ../*.rec.clean.mol2.gz rec.mol2.gz


    if "lig2.smi" not in os.listdir():

      print('Generating SMILES.')
      ! obabel -imol2 lig.mol2.gz -osmi -O lig2.smi &> /dev/null #self docking

    else:

      print('Already SMILEd!')


    insmiles = open('lig2.smi', 'r');
    line=insmiles.readline().strip()
    line2 = line.split()
    ligand_smiles = line2[0]
    insmiles.close();

    lig_src =  'lig.mol2.gz'
    rec_src = 'rec.mol2.gz'

    try:

      REC = Mol2(Input, rec_src)
      RefLig = Mol2(Input, lig_src)
      Lig2 = Mol2()

      ligcheck = Lig2.parse_smiles(Input, ligand_smiles, 'LIG')

    except (RuntimeError, MemoryError):
      print("Skipping, something went wrong...Check the logs.")
      continue


    if (ligcheck):

      Writer = WRITER(Input)
      Coord = COORD_MC()
      HB = FindHB()
      Ene  = Energy2(Input)

      for i in range(len(REC.residue_pointer)-1):
        HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
      HB.find_ligandHB(lig_src, RefLig);

      Dock = Docker(Writer)
      center = Coord.compute_com(RefLig)

      start_time = timeit.default_timer()
      Grids = Grid(Input, Writer)
      Grids.load_grids_from_file()
      time_grid = timeit.default_timer() - start_time
      print(time_grid)


      start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

      grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);


      start_time = timeit.default_timer()
      Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
      time_dock = timeit.default_timer() - start_time


      log = open("log.txt", 'a')

      data = "%.3f\t%.3f\t%.3f\n" %(start_energy,grid_energy, time_dock)
      log.write(data)
      print(data)
      log.close()

  else:
    print('Target already docked!')



The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
#    TARGET  RMSD(Ang)
0
/content/drive/MyDrive/pyLiBELa/SB/1XL2
/content/drive/MyDrive/pyLiBELa/SB/1XL2/self_docking_mma


Already SMILEd!
1.770604983999874
-72.706	13.616	5.168

1
/content/drive/MyDrive/pyLiBELa/SB/1XLX
/content/drive/MyDrive/pyLiBELa/SB/1XLX/self_docking_mma
Target already docked!
2
/content/drive/MyDrive/pyLiBELa/SB/1XLZ
/content/drive/MyDrive/pyLiBELa/SB/1XLZ/self_docking_mma
Target already docked!
3
/content/drive/MyDrive/pyLiBELa/SB/1XM4
/content/drive/MyDrive/pyLiBELa/SB/1XM4/self_docking_mma
Target already docked!
4
/content/drive/MyDrive/pyLiBELa/SB/1XM6
/content/drive/MyDrive/pyLiBELa/SB/1XM6/self_docking_mma
Target already docked!
5
/content/drive/MyDrive/pyLiBELa/SB/1XMU
/content/drive/MyDrive/pyLiBELa/SB/1XMU/self_docking_mma
Target already docked!
6
/content/drive/MyDrive/pyLiBELa/SB/1XMY
/content/drive/MyDrive/pyLiBELa/SB/1XMY/self_docking_mma
Target already docked!
7
/c